# Understanding the trajectory of text data in cancer studies
## Troisieme rapport

Dmytro Zhovtobriukh
\
Anna Novototskykh

In [1]:
import sys
sys.version

'3.11.6 | packaged by conda-forge | (main, Oct  3 2023, 10:40:35) [GCC 12.3.0]'

In [25]:
import torch
torch.cuda.is_available()

True

In [3]:
%pip install signatory==1.2.6.1.9.0 --no-cache-dir --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 107.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for signatory: filename=signatory-1.2.6.1.9.0-cp311-cp311-linux_x86_64.whl size=250332 sha256=8664f40dcced5496bb9f7a7cc0ae05506db94ddff7cc95333ac74d4485da5a80
  Stored in directory: /tmp/pip-ephem-wheel-cache-9ror_4ry/wheels/6a/79/bb/6012413145dd168da55413ef8bc837f507bf829a08a176c329
Successfully built signatory
  Attempting uninstall: signatory
    Found existing installation: signatory 1.2.6.1.9.0
    Uninstalling signatory-1.2.6.1.9.0:
      Successfully uninstalled signatory-1.2.6.1.9.0
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install pandas transformers

Note: you may need to restart the kernel to use updated packages.


### Explore files in dataset

In [5]:
import os

"""# Répertoire actuel
current_directory = os.getcwd()
print("Vous êtes dans le répertoire:", current_directory)

# Accéder au répertoire parent (A)
parent_directory = os.path.dirname(current_directory)

# Changer de répertoire de travail vers le répertoire parent
os.chdir(parent_directory)
# Vérifier le nouveau répertoire
new_directory = os.getcwd()
print("Vous êtes maintenant dans le répertoire:", new_directory)"""

# define whether to use demo dataset or the original
USE_DEMO = False

if USE_DEMO:
    pth = 'C:\\Users\\pc\\Documents\\Recherche\\physionet.org\\files\\mimiciii-demo\\1.4'
    files = [f for f in os.listdir(pth) if f.endswith('.csv')]
else:
    pth = 'data/mimiciii'
    files = [f for f in os.listdir(pth) if f.endswith('.csv.gz')]
print(f'Found {len(files)} files')
print('\n'.join(files))

Found 26 files
DRGCODES.csv.gz
PRESCRIPTIONS.csv.gz
NOTEEVENTS.csv.gz
D_ICD_DIAGNOSES.csv.gz
TRANSFERS.csv.gz
DIAGNOSES_ICD.csv.gz
D_CPT.csv.gz
D_ITEMS.csv.gz
D_ICD_PROCEDURES.csv.gz
MICROBIOLOGYEVENTS.csv.gz
CALLOUT.csv.gz
LABEVENTS.csv.gz
SERVICES.csv.gz
D_LABITEMS.csv.gz
ICUSTAYS.csv.gz
CHARTEVENTS.csv.gz
INPUTEVENTS_CV.csv.gz
CPTEVENTS.csv.gz
ADMISSIONS.csv.gz
DATETIMEEVENTS.csv.gz
OUTPUTEVENTS.csv.gz
PROCEDURES_ICD.csv.gz
INPUTEVENTS_MV.csv.gz
PROCEDUREEVENTS_MV.csv.gz
CAREGIVERS.csv.gz
PATIENTS.csv.gz


### Explore data fields in dataset

In [6]:
import pandas as pd
import numpy as np

Avoid `ParserError: Error tokenizing data. C error: out of memory`

https://stackoverflow.com/questions/41303246/error-tokenizing-data-c-error-out-of-memory-pandas-python-large-file-csv

```python
mylist = []
for chunk in  pd.read_csv('train.csv', chunksize=20000):
    mylist.append(chunk)
big_data = pd.concat(mylist, axis= 0)
del mylist
```

or

```python
big_data = pd.read_csv('train.csv', engine='python')
```

In [7]:
for f in files:
    for d in pd.read_csv(os.path.join(pth, f), low_memory=False, chunksize=200000):
        df = d
        break
    print(f"{f :=^80}")
    print('\n'.join(list(df.columns)))

================================DRGCODES.csv.gz=================================
ROW_ID
SUBJECT_ID
HADM_ID
DRG_TYPE
DRG_CODE
DESCRIPTION
DRG_SEVERITY
DRG_MORTALITY
==============================PRESCRIPTIONS.csv.gz==============================
ROW_ID
SUBJECT_ID
HADM_ID
ICUSTAY_ID
STARTDATE
ENDDATE
DRUG_TYPE
DRUG
DRUG_NAME_POE
DRUG_NAME_GENERIC
FORMULARY_DRUG_CD
GSN
NDC
PROD_STRENGTH
DOSE_VAL_RX
DOSE_UNIT_RX
FORM_VAL_DISP
FORM_UNIT_DISP
ROUTE
===============================NOTEEVENTS.csv.gz================================
ROW_ID
SUBJECT_ID
HADM_ID
CHARTDATE
CHARTTIME
STORETIME
CATEGORY
DESCRIPTION
CGID
ISERROR
TEXT
=============================D_ICD_DIAGNOSES.csv.gz=============================
ROW_ID
ICD9_CODE
SHORT_TITLE
LONG_TITLE
================================TRANSFERS.csv.gz================================
ROW_ID
SUBJECT_ID
HADM_ID
ICUSTAY_ID
DBSOURCE
EVENTTYPE
PREV_CAREUNIT
CURR_CAREUNIT
PREV_WARDID
CURR_WARDID
INTIME
OUTTIME
LOS
==============================DIAGNOSES_ICD.csv.

```
===============================ADMISSIONS.csv.gz================================
ROW_ID
SUBJECT_ID
HADM_ID
ADMITTIME
DISCHTIME
DEATHTIME
ADMISSION_TYPE
ADMISSION_LOCATION
DISCHARGE_LOCATION
INSURANCE
LANGUAGE
RELIGION
MARITAL_STATUS
ETHNICITY
EDREGTIME
EDOUTTIME
DIAGNOSIS
HOSPITAL_EXPIRE_FLAG
HAS_CHARTEVENTS_DATA
=================================CALLOUT.csv.gz=================================
ROW_ID
SUBJECT_ID
HADM_ID
SUBMIT_WARDID
SUBMIT_CAREUNIT
CURR_WARDID
CURR_CAREUNIT
CALLOUT_WARDID
CALLOUT_SERVICE
REQUEST_TELE
REQUEST_RESP
REQUEST_CDIFF
REQUEST_MRSA
REQUEST_VRE
CALLOUT_STATUS
CALLOUT_OUTCOME
DISCHARGE_WARDID
ACKNOWLEDGE_STATUS
CREATETIME
UPDATETIME
ACKNOWLEDGETIME
OUTCOMETIME
FIRSTRESERVATIONTIME
CURRENTRESERVATIONTIME
===============================CAREGIVERS.csv.gz================================
ROW_ID
CGID
LABEL
DESCRIPTION
===============================CHARTEVENTS.csv.gz===============================
ROW_ID
SUBJECT_ID
HADM_ID
ICUSTAY_ID
ITEMID
CHARTTIME
STORETIME
CGID
VALUE
VALUENUM
VALUEUOM
WARNING
ERROR
RESULTSTATUS
STOPPED
================================CPTEVENTS.csv.gz================================
ROW_ID
SUBJECT_ID
HADM_ID
COSTCENTER
CHARTDATE
CPT_CD
CPT_NUMBER
CPT_SUFFIX
TICKET_ID_SEQ
SECTIONHEADER
SUBSECTIONHEADER
DESCRIPTION
=============================DATETIMEEVENTS.csv.gz==============================
ROW_ID
SUBJECT_ID
HADM_ID
ICUSTAY_ID
ITEMID
CHARTTIME
STORETIME
CGID
VALUE
VALUEUOM
WARNING
ERROR
RESULTSTATUS
STOPPED
==============================DIAGNOSES_ICD.csv.gz==============================
ROW_ID
SUBJECT_ID
HADM_ID
SEQ_NUM
ICD9_CODE
================================DRGCODES.csv.gz=================================
ROW_ID
SUBJECT_ID
HADM_ID
DRG_TYPE
DRG_CODE
DESCRIPTION
DRG_SEVERITY
DRG_MORTALITY
==================================D_CPT.csv.gz==================================
ROW_ID
CATEGORY
SECTIONRANGE
SECTIONHEADER
SUBSECTIONRANGE
SUBSECTIONHEADER
CODESUFFIX
MINCODEINSUBSECTION
MAXCODEINSUBSECTION
=============================D_ICD_DIAGNOSES.csv.gz=============================
ROW_ID
ICD9_CODE
SHORT_TITLE
LONG_TITLE
============================D_ICD_PROCEDURES.csv.gz=============================
ROW_ID
ICD9_CODE
SHORT_TITLE
LONG_TITLE
=================================D_ITEMS.csv.gz=================================
ROW_ID
ITEMID
LABEL
ABBREVIATION
DBSOURCE
LINKSTO
CATEGORY
UNITNAME
PARAM_TYPE
CONCEPTID
===============================D_LABITEMS.csv.gz================================
ROW_ID
ITEMID
LABEL
FLUID
CATEGORY
LOINC_CODE
================================ICUSTAYS.csv.gz=================================
ROW_ID
SUBJECT_ID
HADM_ID
ICUSTAY_ID
DBSOURCE
FIRST_CAREUNIT
LAST_CAREUNIT
FIRST_WARDID
LAST_WARDID
INTIME
OUTTIME
LOS
=============================INPUTEVENTS_CV.csv.gz==============================
ROW_ID
SUBJECT_ID
HADM_ID
ICUSTAY_ID
CHARTTIME
ITEMID
AMOUNT
AMOUNTUOM
RATE
RATEUOM
STORETIME
CGID
ORDERID
LINKORDERID
STOPPED
NEWBOTTLE
ORIGINALAMOUNT
ORIGINALAMOUNTUOM
ORIGINALROUTE
ORIGINALRATE
ORIGINALRATEUOM
ORIGINALSITE
=============================INPUTEVENTS_MV.csv.gz==============================
ROW_ID
SUBJECT_ID
HADM_ID
ICUSTAY_ID
STARTTIME
ENDTIME
ITEMID
AMOUNT
AMOUNTUOM
RATE
RATEUOM
STORETIME
CGID
ORDERID
LINKORDERID
ORDERCATEGORYNAME
SECONDARYORDERCATEGORYNAME
ORDERCOMPONENTTYPEDESCRIPTION
ORDERCATEGORYDESCRIPTION
PATIENTWEIGHT
TOTALAMOUNT
TOTALAMOUNTUOM
ISOPENBAG
CONTINUEINNEXTDEPT
CANCELREASON
STATUSDESCRIPTION
COMMENTS_EDITEDBY
COMMENTS_CANCELEDBY
COMMENTS_DATE
ORIGINALAMOUNT
ORIGINALRATE
================================LABEVENTS.csv.gz================================
ROW_ID
SUBJECT_ID
HADM_ID
ITEMID
CHARTTIME
VALUE
VALUENUM
VALUEUOM
FLAG
===========================MICROBIOLOGYEVENTS.csv.gz============================
ROW_ID
SUBJECT_ID
HADM_ID
CHARTDATE
CHARTTIME
SPEC_ITEMID
SPEC_TYPE_DESC
ORG_ITEMID
ORG_NAME
ISOLATE_NUM
AB_ITEMID
AB_NAME
DILUTION_TEXT
DILUTION_COMPARISON
DILUTION_VALUE
INTERPRETATION
===============================NOTEEVENTS.csv.gz================================
ROW_ID
SUBJECT_ID
HADM_ID
CHARTDATE
CHARTTIME
STORETIME
CATEGORY
DESCRIPTION
CGID
ISERROR
TEXT
==============================OUTPUTEVENTS.csv.gz===============================
ROW_ID
SUBJECT_ID
HADM_ID
ICUSTAY_ID
CHARTTIME
ITEMID
VALUE
VALUEUOM
STORETIME
CGID
STOPPED
NEWBOTTLE
ISERROR
================================PATIENTS.csv.gz=================================
ROW_ID
SUBJECT_ID
GENDER
DOB
DOD
DOD_HOSP
DOD_SSN
EXPIRE_FLAG
==============================PRESCRIPTIONS.csv.gz==============================
ROW_ID
SUBJECT_ID
HADM_ID
ICUSTAY_ID
STARTDATE
ENDDATE
DRUG_TYPE
DRUG
DRUG_NAME_POE
DRUG_NAME_GENERIC
FORMULARY_DRUG_CD
GSN
NDC
PROD_STRENGTH
DOSE_VAL_RX
DOSE_UNIT_RX
FORM_VAL_DISP
FORM_UNIT_DISP
ROUTE
===========================PROCEDUREEVENTS_MV.csv.gz============================
ROW_ID
SUBJECT_ID
HADM_ID
ICUSTAY_ID
STARTTIME
ENDTIME
ITEMID
VALUE
VALUEUOM
LOCATION
LOCATIONCATEGORY
STORETIME
CGID
ORDERID
LINKORDERID
ORDERCATEGORYNAME
SECONDARYORDERCATEGORYNAME
ORDERCATEGORYDESCRIPTION
ISOPENBAG
CONTINUEINNEXTDEPT
CANCELREASON
STATUSDESCRIPTION
COMMENTS_EDITEDBY
COMMENTS_CANCELEDBY
COMMENTS_DATE
=============================PROCEDURES_ICD.csv.gz==============================
ROW_ID
SUBJECT_ID
HADM_ID
SEQ_NUM
ICD9_CODE
================================SERVICES.csv.gz=================================
ROW_ID
SUBJECT_ID
HADM_ID
TRANSFERTIME
PREV_SERVICE
CURR_SERVICE
================================TRANSFERS.csv.gz================================
ROW_ID
SUBJECT_ID
HADM_ID
ICUSTAY_ID
DBSOURCE
EVENTTYPE
PREV_CAREUNIT
CURR_CAREUNIT
PREV_WARDID
CURR_WARDID
INTIME
OUTTIME
LOS
```

### Explore data rows

In [8]:
for f in files:
    for d in pd.read_csv(os.path.join(pth, f), low_memory=False, chunksize=200000):
        df = d
        break
    print(f"{f :=^80}")
    print(df.head(1))

================================DRGCODES.csv.gz=================================
   ROW_ID  SUBJECT_ID  HADM_ID DRG_TYPE  DRG_CODE  \
0     342        2491   144486     HCFA        28   

                                         DESCRIPTION  DRG_SEVERITY  \
0  TRAUMATIC STUPOR & COMA, COMA <1 HR AGE >17 WI...           NaN   

   DRG_MORTALITY  
0            NaN  
==============================PRESCRIPTIONS.csv.gz==============================
    ROW_ID  SUBJECT_ID  HADM_ID  ICUSTAY_ID            STARTDATE  \
0  2214776           6   107064         NaN  2175-06-11 00:00:00   

               ENDDATE DRUG_TYPE        DRUG DRUG_NAME_POE DRUG_NAME_GENERIC  \
0  2175-06-12 00:00:00      MAIN  Tacrolimus    Tacrolimus        Tacrolimus   

  FORMULARY_DRUG_CD     GSN          NDC PROD_STRENGTH DOSE_VAL_RX  \
0             TACR1  021796  469061711.0   1mg Capsule           2   

  DOSE_UNIT_RX FORM_VAL_DISP FORM_UNIT_DISP ROUTE  
0           mg             2            CAP    PO  
=========

```
=================================ADMISSIONS.csv=================================
   row_id  subject_id  hadm_id            admittime            dischtime  \
0   12258       10006   142345  2164-10-23 21:09:00  2164-11-01 17:15:00   

  deathtime admission_type    admission_location discharge_location insurance  \
0       NaN      EMERGENCY  EMERGENCY ROOM ADMIT   HOME HEALTH CARE  Medicare   

  language  religion marital_status               ethnicity  \
0      NaN  CATHOLIC      SEPARATED  BLACK/AFRICAN AMERICAN   

             edregtime            edouttime diagnosis  hospital_expire_flag  \
0  2164-10-23 16:43:00  2164-10-23 23:00:00    SEPSIS                     0   

   has_chartevents_data  
0                     1  
==================================CALLOUT.csv===================================
   row_id  subject_id  hadm_id  submit_wardid submit_careunit  curr_wardid  \
0    3917       10017   199207              7             NaN           45   

  curr_careunit  callout_wardid callout_service  request_tele  ...  \
0           CCU               1             MED             1  ...   

   callout_status  callout_outcome  discharge_wardid  acknowledge_status  \
0        Inactive       Discharged              45.0        Acknowledged   

            createtime           updatetime      acknowledgetime  \
0  2149-05-31 10:44:34  2149-05-31 10:44:34  2149-05-31 15:08:04   

           outcometime firstreservationtime currentreservationtime  
0  2149-05-31 22:40:02                  NaN                    NaN  

[1 rows x 24 columns]
=================================CAREGIVERS.csv=================================
   row_id   cgid label description
0    2228  16174    RO   Read Only
================================CHARTEVENTS.csv=================================
    row_id  subject_id  hadm_id  icustay_id  itemid            charttime  \
0  5279021       40124   126179    279554.0  223761  2130-02-04 04:00:00   

             storetime   cgid value  valuenum valueuom  warning  error  \
0  2130-02-04 04:35:00  19085  95.9      95.9       ?F      0.0    0.0   

  resultstatus stopped  
0          NaN     NaN  
=================================CPTEVENTS.csv==================================
   row_id  subject_id  hadm_id costcenter chartdate  cpt_cd  cpt_number  \
0    4615       10117   105150        ICU       NaN   99254       99254   

   cpt_suffix  ticket_id_seq              sectionheader subsectionheader  \
0         NaN            1.0  Evaluation and management    Consultations   

  description  
0         NaN  
===============================DATETIMEEVENTS.csv===============================
   row_id  subject_id  hadm_id  icustay_id  itemid            charttime  \
0  208474       10076   198503    201006.0    5684  2107-03-25 04:00:00   

             storetime   cgid                value valueuom  warning  error  \
0  2107-03-25 04:34:00  20482  2107-03-24 00:00:00     Date      NaN    NaN   

   resultstatus   stopped  
0           NaN  NotStopd  
===============================DIAGNOSES_ICD.csv================================
   row_id  subject_id  hadm_id  seq_num icd9_code
0  112344       10006   142345        1     99591
==================================DRGCODES.csv==================================
   row_id  subject_id  hadm_id drg_type  drg_code  \
0    1338       10130   156668     HCFA       148   

                                         description  drg_severity  \
0  MAJOR SMALL & LARGE BOWEL PROCEDURES WITH COMP...           NaN   

   drg_mortality  
0            NaN  
===================================D_CPT.csv====================================
   row_id  category sectionrange              sectionheader subsectionrange  \
0       1         1  99201-99499  Evaluation and management     99201-99216   

                   subsectionheader codesuffix  mincodeinsubsection  \
0  Office/other outpatient services        NaN                99201   

   maxcodeinsubsection  
0                99216  
==============================D_ICD_DIAGNOSES.csv===============================
   row_id icd9_code              short_title  \
0       1     01716  Erythem nod tb-oth test   

                                          long_title  
0  Erythema nodosum with hypersensitivity reactio...  
==============================D_ICD_PROCEDURES.csv==============================
   row_id  icd9_code               short_title  \
0       1       1423  Chorioret les xenon coag   

                                          long_title  
0  Destruction of chorioretinal lesion by xenon a...  
==================================D_ITEMS.csv===================================
   row_id  itemid               label abbreviation dbsource      linksto  \
0       1    1435  Sustained Nystamus          NaN  carevue  chartevents   

  category unitname param_type  conceptid  
0      NaN      NaN        NaN        NaN  
=================================D_LABITEMS.csv=================================
   row_id  itemid          label  fluid   category loinc_code
0       1   50800  SPECIMEN TYPE  BLOOD  BLOOD GAS        NaN
==================================ICUSTAYS.csv==================================
   row_id  subject_id  hadm_id  icustay_id dbsource first_careunit  \
0   12742       10006   142345      206504  carevue           MICU   

  last_careunit  first_wardid  last_wardid               intime  \
0          MICU            52           52  2164-10-23 21:10:15   

               outtime     los  
0  2164-10-25 12:21:07  1.6325  
===============================INPUTEVENTS_CV.csv===============================
   row_id  subject_id  hadm_id  icustay_id            charttime  itemid  \
0    1184       10114   167957      234989  2171-11-03 15:00:00   30056   

   amount amountuom  rate rateuom  ...  orderid  linkorderid  stopped  \
0   400.0        ml   NaN     NaN  ...  2557279      2557279      NaN   

   newbottle originalamount  originalamountuom  originalroute originalrate  \
0        NaN            NaN                 ml           Oral          NaN   

  originalrateuom  originalsite  
0             NaN           NaN  

[1 rows x 22 columns]
===============================INPUTEVENTS_MV.csv===============================
   row_id  subject_id  hadm_id  icustay_id            starttime  \
0  118897       42367   139932      250305  2147-10-29 16:45:00   

               endtime  itemid  amount amountuom  rate  ... totalamountuom  \
0  2147-10-29 16:46:00  225799    60.0        ml   NaN  ...             ml   

  isopenbag  continueinnextdept  cancelreason  statusdescription  \
0         0                   0             0    FinishedRunning   

  comments_editedby comments_canceledby comments_date originalamount  \
0               NaN                 NaN           NaN           60.0   

   originalrate  
0          60.0  

[1 rows x 31 columns]
=================================LABEVENTS.csv==================================
    row_id  subject_id  hadm_id  itemid            charttime value  valuenum  \
0  6244563       10006      NaN   50868  2164-09-24 20:21:00    19      19.0   

  valueuom flag  
0    mEq/L  NaN  
=============================MICROBIOLOGYEVENTS.csv=============================
   row_id  subject_id  hadm_id            chartdate            charttime  \
0  134694       10006   142345  2164-10-23 00:00:00  2164-10-23 15:30:00   

   spec_itemid spec_type_desc  org_itemid                            org_name  \
0        70012  BLOOD CULTURE     80155.0  STAPHYLOCOCCUS, COAGULASE NEGATIVE   

   isolate_num  ab_itemid ab_name dilution_text dilution_comparison  \
0          2.0        NaN     NaN           NaN                 NaN   

   dilution_value interpretation  
0             NaN            NaN  
=================================NOTEEVENTS.csv=================================
Empty DataFrame
Columns: [row_id, subject_id, hadm_id, chartdate, charttime, storetime, category, description, cgid, iserror, text]
Index: []
================================OUTPUTEVENTS.csv================================
   row_id  subject_id  hadm_id  icustay_id            charttime  itemid  \
0    6540       10114   167957    234989.0  2171-10-30 20:00:00   40055   

   value valueuom            storetime   cgid  stopped  newbottle  iserror  
0   39.0       ml  2171-10-30 20:38:00  15029      NaN        NaN      NaN  
==================================PATIENTS.csv==================================
   row_id  subject_id gender                  dob                  dod  \
0    9467       10006      F  2094-03-05 00:00:00  2165-08-12 00:00:00   

              dod_hosp              dod_ssn  expire_flag  
0  2165-08-12 00:00:00  2165-08-12 00:00:00            1  
===============================PRESCRIPTIONS.csv================================
   row_id  subject_id  hadm_id  icustay_id            startdate  \
0   32600       42458   159647         NaN  2146-07-21 00:00:00   

               enddate drug_type                         drug  \
0  2146-07-22 00:00:00      MAIN  Pneumococcal Vac Polyvalent   

                 drug_name_poe            drug_name_generic formulary_drug_cd  \
0  Pneumococcal Vac Polyvalent  PNEUMOcoccal Vac Polyvalent           PNEU25I   

       gsn        ndc     prod_strength dose_val_rx dose_unit_rx  \
0  48548.0  6494300.0  25mcg/0.5mL Vial         0.5           mL   

  form_val_disp form_unit_disp route  
0             1           VIAL    IM  
=============================PROCEDUREEVENTS_MV.csv=============================
   row_id  subject_id  hadm_id  icustay_id            starttime  \
0    8641       42367   139932      250305  2147-10-03 16:40:00   

               endtime  itemid  value valueuom        location  ...  \
0  2147-10-06 20:00:00  224263   4520      min  Right Femoral.  ...   

  ordercategoryname secondaryordercategoryname  ordercategorydescription  \
0    Invasive Lines                        NaN                      Task   

   isopenbag  continueinnextdept cancelreason  statusdescription  \
0          1                   0            0    FinishedRunning   

  comments_editedby  comments_canceledby  comments_date  
0               NaN                  NaN            NaN  

[1 rows x 25 columns]
===============================PROCEDURES_ICD.csv===============================
   row_id  subject_id  hadm_id  seq_num  icd9_code
0    3994       10114   167957        1       3605
==================================SERVICES.csv==================================
   row_id  subject_id  hadm_id         transfertime prev_service curr_service
0   14974       10006   142345  2164-10-23 21:10:15          NaN          MED
=================================TRANSFERS.csv==================================
   row_id  subject_id  hadm_id  icustay_id dbsource eventtype prev_careunit  \
0   54440       10006   142345    206504.0  carevue     admit           NaN   

  curr_careunit  prev_wardid  curr_wardid               intime  \
0          MICU          NaN         52.0  2164-10-23 21:10:15   

               outtime    los  
0  2164-10-25 12:21:07  39.18  
```

# Importer les données et les packages

Dans cette section, nous incluons les bibliothèques nécessaires et chargeons les données à partir de fichiers CSV dans des DataFrames Pandas. Nous effectuons également des transformations sur les types de données et procédons au nettoyage des données en vue d'analyses futures.

Voici le but d'utilisation de modules:
* Pandas - le module pour gérer facilement les tables (dataframes)
* Numpy - le module de base pour la plupart de modules scientifiques


In [3]:
# Importer les bibliothèques nécessaires
import pandas as pd
import numpy as np
import random

In [9]:
data = pd.concat([chunk for chunk in pd.read_csv(os.path.join(pth, 'NOTEEVENTS.csv.gz'), chunksize=20000)], axis=0)
len(data)

2083180

In [ ]:
print(data.iloc[0,:])

In [ ]:
print(data.iloc[0,:]['TEXT'])

In [10]:
# Lire les données à partir d'un fichier CSV dans un DataFrame Pandas
#data = pd.read_csv("/kaggle/input/clean-data/clean_data.csv", low_memory=False)

# Convertir les colonnes 'SUBJECT_ID' et 'HADM_ID' en chaînes de caractères
data['SUBJECT_ID'] = data['SUBJECT_ID'].astype(str)
data['HADM_ID'] = data['HADM_ID'].astype(str)

# Le nettoyage de données. Remplacer la chaîne "nan" par des valeurs NaN réelles dans la colonne 'HADM_ID'
data['HADM_ID'] = data['HADM_ID'].replace("nan", np.nan)

# Convertir la colonne 'CHARTTIME' qui contient les timestamps en un format datetime avec le format spécifié
data['CHARTTIME'] = pd.to_datetime(data['CHARTTIME'], format='%Y-%m-%d %H:%M:%S')

# Supprimer les lignes ayant des valeurs manquantes dans la colonne 'HADM_ID'
data = data.dropna(subset=["HADM_ID"])

# Nettoyer le dataframe de champs nulles par supprimant les deux derniers caractères de la colonne 'HADM_ID'
data["HADM_ID"] = data["HADM_ID"].str[:-2]

# Convertir la colonne 'CHARTDATE' qui contient les timestamps en un format datetime
data['CHARTDATE'] = pd.to_datetime(data['CHARTDATE'])

# Convertir la colonne 'TIME' en entiers
# data['TIME'] = data['TIME'].astype(int)
data['TIME'] = 0

In [11]:
adm = pd.concat([chunk for chunk in pd.read_csv(os.path.join(pth, 'ADMISSIONS.csv.gz'), chunksize=20000)], axis=0)
len(adm)

58976

In [13]:
adm

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,58594,98800,191113,2131-03-30 21:13:00,2131-04-02 15:02:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE,2131-03-30 19:44:00,2131-03-30 22:41:00,TRAUMA,0,1
58972,58595,98802,101071,2151-03-05 20:00:00,2151-03-06 09:10:00,2151-03-06 09:10:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,2151-03-05 17:23:00,2151-03-05 21:06:00,SAH,1,1
58973,58596,98805,122631,2200-09-12 07:15:00,2200-09-20 12:08:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,NaN,NaN,RENAL CANCER/SDA,0,1
58974,58597,98813,170407,2128-11-11 02:29:00,2128-12-22 13:11:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,ENGL,CATHOLIC,MARRIED,WHITE,2128-11-10 23:48:00,2128-11-11 03:16:00,S/P FALL,0,0


In [12]:
# Lire les données d'admission à partir d'un autre fichier CSV dans un DataFrame Pandas
# adm = pd.read_csv("/kaggle/input/clean-data/clean_adm.csv", low_memory=False)

# Convertir les colonnes 'SUBJECT_ID' et 'HADM_ID' en chaînes de caractères
adm['SUBJECT_ID'] = adm['SUBJECT_ID'].astype(str)
adm['HADM_ID'] = adm['HADM_ID'].astype(str)

# Convertir la colonne 'HOSPITAL_EXPIRE_FLAG' en entiers
adm['HOSPITAL_EXPIRE_FLAG'] = adm['HOSPITAL_EXPIRE_FLAG'].astype(int)

# Convertir les colonnes 'ADMITTIME' et 'DISCHTIME' en un format datetime avec le format spécifié
adm['ADMTTIME'] = pd.to_datetime(adm['ADMITTIME'], format='%Y-%m-%d %H:%M:%S')
adm['DISCHTIME'] = pd.to_datetime(adm['DISCHTIME'], format='%Y-%m-%d %H:%M:%S')

# Filtrer les données d'admission pour inclure uniquement les lignes avec des valeurs 'SUBJECT_ID' présentes dans le DataFrame 'data'
adm = adm[adm["SUBJECT_ID"].isin(data["SUBJECT_ID"].unique())]

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

NameError: name 'torch' is not defined

# Créer les jeux de test et d'entraînement

Cette section concerne la création des ensembles de données utilisés pour l'apprentissage et les tests. Nous regroupons les données relatives aux patients selon leurs identifiants uniques, puis nous les étiquetons en fonction de la présence ou non d'une condition spécifique. Cette étape permet ainsi la formation de l'ensemble d'apprentissage. De plus, nous sélectionnons aléatoirement un sous-ensemble de patients pour constituer l'ensemble de test.

Le but de cet étape est la division le jeu de données pour laisser entrainer le modèle et ainsi le évaluer. La division raisonnable est crucial pour obtenir un vrai metrique de modèle et aussi éviter "overfitting" ou "underfitting".

Puis, le modèle s'entrainera sur le jeu d'entrainement et après on calcule la metrique sur le jeu de test. Ce metrique montre comment notre modèle marche sur les données reéls et ainsi on peut comparer les modèles differentes.

In [ ]:
# Créer un dataframe vide avec deux colonnes pour le nouveau dataframe
new_data = pd.DataFrame(columns=["SUBJECT_ID", "HOSPITAL_EXPIRE_FLAG"])

# Grouper les données d'admission par "SUBJECT_ID"
grouped_adm = adm.groupby("SUBJECT_ID")
print(len(grouped_adm))

# Parcourir chaque groupe de données associées à un "SUBJECT_ID"
for subject_id, group in grouped_adm:
    # Vérifier si le groupe contient au moins un enregistrement avec la valeur 1 dans la colonne "HOSPITAL_EXPIRE_FLAG"
    if group["HOSPITAL_EXPIRE_FLAG"].eq(1).any():
        # Si oui, ajouter le "SUBJECT_ID" et la valeur 1 dans le nouveau dataframe
        new_data = pd.concat([new_data, pd.DataFrame({"SUBJECT_ID": [subject_id], "HOSPITAL_EXPIRE_FLAG": [1]})], ignore_index=True)
    else:
        # Sinon, ajouter le "SUBJECT_ID" et la valeur 0 dans le nouveau dataframe
        new_data = pd.concat([new_data, pd.DataFrame({"SUBJECT_ID": [subject_id], "HOSPITAL_EXPIRE_FLAG": [0]})], ignore_index=True)

# Créer un nouveau dataframe avec les "SUBJECT_ID" ayant la valeur 1 dans la colonne "HOSPITAL_EXPIRE_FLAG"
label_1 = new_data[new_data["HOSPITAL_EXPIRE_FLAG"] == 1].reset_index()
print(len(label_0))

# Sélectionner aléatoirement le même nombre de "SUBJECT_ID" ayant la valeur 0
label_0 = new_data[new_data["HOSPITAL_EXPIRE_FLAG"] == 0].reset_index()
print(len(label_1))

# Fonction pour diviser les documents en plus petits morceaux

Dans cette partie, nous définissons une fonction permettant de découper les documents textuels en segments plus petits afin de faciliter leur traitement ultérieur.

In [ ]:
def split_text(text, k):
    # Convertir le texte en une liste de mots
    words = text.split()

    # Déterminer le nombre total de mots dans le texte
    num_words = len(words)

    # Calculer le nombre de mots par partie
    words_per_part = num_words // k

    # Calculer le nombre de mots restants si num_words n'est pas un multiple de k
    remainder = num_words % k

    # Initialiser une liste pour stocker les parties découpées du texte
    parts = []

    # Initialiser l'indice de début pour la découpe
    start = 0

    # Parcourir chaque partie
    for i in range(k):
        # Calculer la position de fin pour la i-ème partie
        end = start + words_per_part + (i < remainder)
        # La variable "end" correspond à la position du dernier mot de la i-ème partie

        # Ajouter la partie actuelle à la liste des parties
        parts.append(words[start:end])

        # Mettre à jour l'indice de début pour la prochaine partie
        start = end

    # Convertir les listes de mots en chaînes de caractères
    parts = [" ".join(part) for part in parts]

    return parts

In [ ]:
device = torch.device('cuda')
torch.cuda.set_device(0)
print('Using device:', device)
print()

# Jeux de données d'entraînement

Cette section prépare l'ensemble de données d'apprentissage en extrayant et traitant les embeddings ClinicalBERT à partir des informations relatives aux patients. Ces embeddings sont ensuite organisés en vue d'analyses ultérieures.

## [TRAIN] Charger le modèle ClinicalBERT depuis Hugging Face

Dans cette partie, nous chargeons le modèle ClinicalBERT à partir du référentiel Hugging Face et sélectionnons un échantillon spécifique de patients pour l'apprentissage.

In [ ]:
# Importer les bibliothèques nécessaires
from transformers import AutoTokenizer, AutoModel
#import torch
from torch import nn

# Charger le modèle de langue pré-entraîné (Bio_ClinicalBERT) et le tokenizer associé
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT").to("cuda")

# Sélectionner aléatoirement 1000 individus de chaque classe (label_1 et label_0)
sample = pd.concat([label_1.sample(n=1000, random_state=56), label_0.sample(n=1000, random_state=78)]).reset_index().drop('index', axis=1)

# Filtrer le dataframe de données en ne conservant que les patients sélectionnés précédemment
filtered_data = data[data["SUBJECT_ID"].isin(sample["SUBJECT_ID"].values)]

# Regrouper les données filtrées par 'SUBJECT_ID' en agrégeant les listes de 'TEXT' et 'TIME'
grouped_sample = filtered_data.groupby('SUBJECT_ID').agg({'TEXT': list, 'TIME': list}).reset_index()

## [TRAIN] Extraire les tokens CLS

Dans cette partie, le code traite les données textuelles en les divisant en segments plus petits et extrait les embeddings ClinicalBERT correspondants à ces segments. Le résultat est un dictionnaire qui associe chaque patient à ses embeddings ClinicalBERT.

In [ ]:
# Créer un dictionnaire de la forme {n° patient: [liste des textes, valeurs time]} pour faciliter l'itération
grouped_texts_dict = grouped_sample.set_index('SUBJECT_ID')[['TEXT', 'TIME']].to_dict(orient='index')

# Initialiser une liste pour stocker les valeurs 'TIME' de chaque partie d'un document
time_list = []

# Initialiser un dictionnaire pour stocker les embeddings
embeddings_dict = {}

# Parcourir les patients et leurs données associées
for subject_id, values in grouped_texts_dict.items():
    texts = values['TEXT']  # Récupérer la liste des documents
    times = values['TIME']  # Récupérer la liste des valeurs 'TIME' associées aux documents
    embeddings_list = []  # Liste pour stocker les embeddings de toutes les parties de tous les documents

    # Parcourir les documents et leurs valeurs 'TIME' associées
    for text, time in zip(texts, times):
        # Diviser le texte en parties égales
        encoded_text = tokenizer.encode(text)  # Encodage du texte en une séquence de tokens
        n_tokens = len(encoded_text)  # Nombre de tokens dans la séquence
        n_chunks = max(1, n_tokens // 512)  # Calcul du nombre optimal de parties
        parties = split_text(text, n_chunks)  # Liste des parties du texte

        # Stocker les embeddings des différentes parties du document
        cls_embeddings_list = []  # Liste pour stocker les embeddings [CLS] des parties

        # Parcourir les parties du document
        for partie in parties:
            # Convertir la partie dans un format compatible avec le modèle
            inputs = tokenizer(partie, return_tensors='pt', padding=True, truncation=True, max_length=512).to("cuda")

            with torch.no_grad():
                # Effectuer l'inférence pour obtenir les résultats du modèle
                outputs = model(**inputs)

            # Récupérer l'embedding du token [CLS] pour chaque partie
            cls_embeddings = outputs.last_hidden_state[:, 0, :]
            # Stocker l'embedding dans la liste
            cls_embeddings_list.append(cls_embeddings)
            # Stocker la valeur 'TIME' (la même pour toutes les parties du même document)
            time_list.append(time)

        # Ajouter la liste des embeddings [CLS] à la liste des embeddings de ce document
        embeddings_list += cls_embeddings_list

    # Stocker les embeddings dans un dictionnaire avec le numéro du patient comme clé
    embeddings_dict[subject_id] = torch.stack(embeddings_list)


## [TRAIN] Réduction de dimension

Ici, le code se concentre sur la réduction de la dimensionnalité des embeddings obtenus lors de l'étape précédente. Il utilise une technique appelée projection gaussienne aléatoire pour transformer les embeddings dans un espace de dimension inférieure, ce qui rend les données plus gérables et peut potentiellement améliorer les performances du modèle. Le résultat est un dictionnaire contenant les embeddings réduits pour chaque patient.

Le but de cette action c'est de faire plus simple le modele donc il demande moins de ressources pour entrainer.

### [TRAIN] Projection gaussienne aléatoire

In [ ]:
%pip install scikit-learn

In [ ]:
# Importer la classe random_projection du module sklearn
from sklearn import random_projection

# ClinicalBERT renvoie des embeddings au format de tensor PyTorch.
# Nous les convertissons en tableau NumPy pour la réduction de dimension
flattened_embeddings_dict = {key: tensor.view(tensor.shape[0], -1).cpu().numpy() for key, tensor in embeddings_dict.items()}

# Créer une liste pour stocker le nombre d'embeddings que possède chaque patient
lengths = [len(tensor) for tensor in flattened_embeddings_dict.values()]

# Concaténer tous les embeddings pour créer une matrice unique
embeddings_np = np.concatenate(list(flattened_embeddings_dict.values()))

# Initialiser la projection gaussienne aléatoire avec 100 composantes
transformer = random_projection.GaussianRandomProjection(n_components=100)

# Réduire la dimension des embeddings en utilisant la projection gaussienne aléatoire
reduced_embeddings_np = transformer.fit_transform(embeddings_np)

# Diviser les embeddings réduits pour chaque patient
reduced_embeddings_list = np.split(reduced_embeddings_np, np.cumsum(lengths)[:-1])

# Recréer le dictionnaire des embeddings réduits avec les numéros de patient correspondants
reduced_embeddings_dict = {key: tensor for key, tensor in zip(flattened_embeddings_dict.keys(), reduced_embeddings_list)}

# Filtrer les embeddings dont la forme est différente de (1, 100)
filtered_embeddings_dict = {key: value for key, value in reduced_embeddings_dict.items() if value.shape != (1, 100)}
reduced_embeddings_dict = filtered_embeddings_dict
del filtered_embeddings_dict  # Supprimer la variable temporaire

# Ajouter la valeur 'time' à la fin de chaque embedding
for key, time in zip(reduced_embeddings_dict.keys(), time_list):
    array_vide = np.empty((reduced_embeddings_dict[key].shape[0], 101))
    for i in range(reduced_embeddings_dict[key].shape[0]):
        array = np.append(reduced_embeddings_dict[key][i], time)
        array_vide[i:] = array
    reduced_embeddings_dict[key] = array_vide

# Vérifier que le lien entre chaque sujet et ses embeddings est conservé
for key in reduced_embeddings_dict.keys():
    print(f"Subject ID: {key}, Embedding shape: {reduced_embeddings_dict[key].shape}")
    break

### [TRAIN] ACP

Dans cette section, nous appliquons une Analyse en Composantes Principales (ACP) aux embeddings. L'objectif de l'ACP est de réduire davantage la dimensionnalité des données tout en préservant autant d'informations que possible. Le code calcule la variance expliquée par chaque composante principale, ce qui permet d'évaluer l'efficacité de la réduction de dimension.

In [ ]:
# Importer la classe PCA (Analyse en Composantes Principales) du module sklearn
from sklearn.decomposition import PCA

# Créer un nouveau dictionnaire pour stocker les embeddings aplatis
flattened_embeddings_dict = {key: tensor.view(tensor.shape[0], -1).cpu().numpy() for key, tensor in embeddings_dict.items()}

# Créer une liste pour stocker le nombre d'embeddings pour chaque patient
lengths = [len(tensor) for tensor in flattened_embeddings_dict.values()]

# Concaténer tous les embeddings en une seule matrice
embeddings_np = np.concatenate(list(flattened_embeddings_dict.values()))

# Initialiser le modèle PCA (Analyse en Composantes Principales) avec 100 composantes
pca = PCA(n_components=100)

# Ajuster le modèle PCA aux données et les transformer pour réduire la dimension
reduced_embeddings_np = pca.fit_transform(embeddings_np)

# Séparer les embeddings transformés pour chaque sujet
reduced_embeddings_list = np.split(reduced_embeddings_np, np.cumsum(lengths)[:-1])

# Recréer le dictionnaire des embeddings transformés avec les clés correspondantes
reduced_embeddings_dict = {key: tensor for key, tensor in zip(flattened_embeddings_dict.keys(), reduced_embeddings_list)}

# Filtrer les embeddings dont la forme est différente de (1, 100)
filtered_embeddings_dict = {key: value for key, value in reduced_embeddings_dict.items() if value.shape != (1, 100)}
reduced_embeddings_dict = filtered_embeddings_dict
del filtered_embeddings_dict  # Supprimer la variable temporaire

# Ajouter la valeur 'time' à la fin de chaque embedding
for key, time in zip(reduced_embeddings_dict.keys(), time_list):
    # Créer un tableau vide pour stocker les embeddings avec le temps
    array_vide = np.empty((reduced_embeddings_dict[key].shape[0], 101))
    for i in range(reduced_embeddings_dict[key].shape[0]):
        # Ajouter la valeur 'time' à la fin de chaque embedding
        array = np.append(reduced_embeddings_dict[key][i], time)
        array_vide[i:] = array
    reduced_embeddings_dict[key] = array_vide

# Afficher la variance expliquée par chaque composante principale
print("Variance expliquée par chaque composante principale:", pca.explained_variance_ratio_)

# Afficher la variance totale expliquée par toutes les composantes principales
print("Variance totale expliquée:", sum(pca.explained_variance_ratio_))

# Vérifier que le lien entre chaque sujet et ses embeddings est bien conservé
for key in reduced_embeddings_dict.keys():
    print(f"ID du patient : {key}, Forme des embeddings : {reduced_embeddings_dict[key].shape}")
    break

## [TRAIN] Calculer les signatures

Cette partie du code calcule les signatures logarithmiques pour les embeddings réduits.
Les signatures logarithmiques capturent des informations plus complexes dans les données, ce qui peut être très utile lors de l'entraînement d'un modèle de prédiction. Cela aboutit à la création d'un dictionnaire où chaque patient est représenté par des plongements sous forme de signatures logarithmiques.

In [ ]:
# Importer les bibliothèques nécessaires
#import torch
import signatory

# Ordre de la signature tronquée
depth = 2

# Créer un nouveau dictionnaire pour stocker les résultats de la log signature
log_signature_dict = {}

# Parcourir le dictionnaire des embeddings réduits (reduced_embeddings_dict)
for key, value in reduced_embeddings_dict.items():
    # Convertir les tableaux NumPy en tenseurs PyTorch de type float
    tensor = torch.from_numpy(value).float().to("cuda")

    # Ajouter une dimension "batch" pour correspondre au format requis (batch, stream, channel)
    tensor = tensor.unsqueeze(0).to("cuda")

    # Calculer la log signature en utilisant la bibliothèque Signatory
    log_signature = signatory.logsignature(path=tensor, depth=depth)

    # Enlever la dimension "batch" que nous avons ajoutée précédemment
    log_signature = log_signature.squeeze(0).to("cuda")

    # Ajouter le résultat dans le dictionnaire log_signature_dict
    log_signature_dict[key] = log_signature

# À ce stade, log_signature_dict est un dictionnaire où chaque clé correspond à un numéro de patient, et chaque valeur est la log signature de ce patient.


In [ ]:
print(log_signature_dict)

## [TRAIN] Dataframe utilisé pour l'entraînement

Après avoir effectué l'extraction, la réduction de dimension et le calcul des signatures logarithmiques pour les embeddings, le code transforme les résultats en un DataFrame Pandas.

In [ ]:
# Convertir le dictionnaire log_signature_dict en un DataFrame
df_features = pd.DataFrame.from_dict(log_signature_dict, orient='index')

# Réinitialiser l'index pour que 'SUBJECT_ID' devienne une colonne du DataFrame
df_features.reset_index(inplace=True)

# Renommer la colonne d'index en 'SUBJECT_ID'
df_features.rename(columns={'index':'SUBJECT_ID'}, inplace=True)

# Parcourir chaque colonne du DataFrame et convertir chaque tenseur en float (si nécessaire)
for col in df_features.columns:
    df_features[col] = df_features[col].apply(lambda x: x.item() if torch.is_tensor(x) else x)

# Fusionner le DataFrame df_features avec le DataFrame new_data sur la colonne 'SUBJECT_ID'
df_final = pd.merge(df_features, new_data[['SUBJECT_ID', 'HOSPITAL_EXPIRE_FLAG']], on='SUBJECT_ID', how='inner')




In [ ]:
# Afficher la forme (nombre de lignes et de colonnes) du DataFrame df_final
df_final.shape

In [ ]:
df_final.head()

# Jeux de données test

Cette partie prépare un ensemble de données distinct pour les tests en choisissant au hasard un groupe de patients avec des étiquettes équilibrées de décès à l'hôpital. Ces données seront utilisées pour évaluer les performances du modèle sur de nouvelles observations.

## [TEST] Charger le modèle ClinicalBERT depuis Hugging Face

Dans cette partie, le code charge le modèle ClinicalBERT ainsi que son tokenizer depuis la bibliothèque Hugging Face. Ces éléments sont indispensables pour extraire les représentations vectorielles à partir des informations textuelles des patients dans l'ensemble de données de test. Cela permet d'avoir accès au modèle pré-entraîné afin de traiter les données de test.

In [ ]:
# Importer les bibliothèques nécessaires
from transformers import AutoTokenizer, AutoModel
#import torch
from torch import nn

# Charger le modèle pré-entraîné Bio_ClinicalBERT et son tokenizer
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT").to("cuda")

# Filtrer les données test (new_data) pour ne conserver que les patients absents dans le dictionnaire des embeddings (flattened_embeddings_dict)
new_data_test = new_data[~new_data["SUBJECT_ID"].isin(flattened_embeddings_dict.keys())]

# Sélectionner les données test ayant la valeur "HOSPITAL_EXPIRE_FLAG" égale à 1 (décédés)
label_1 = new_data_test[new_data_test["HOSPITAL_EXPIRE_FLAG"] == 1].reset_index()

# Sélectionner aléatoirement le même nombre de "SUBJECT_ID" ayant la valeur "HOSPITAL_EXPIRE_FLAG" égale à 0 (non décédés)
label_0 = new_data_test[new_data_test["HOSPITAL_EXPIRE_FLAG"] == 0].reset_index()

# Créer un échantillon test en combinant les données décédées (150 patients) et non décédées (850 patients)
sample_test = pd.concat([label_1.sample(n=150, random_state=56), label_0.sample(n=850, random_state=78)]).reset_index().drop('index', axis=1)

# Filtrer les données d'observation (data) pour ne conserver que les patients présents dans l'échantillon test (sample_test)
filtered_data_test = data[data["SUBJECT_ID"].isin(sample_test["SUBJECT_ID"].values)]

# Regrouper les données test par "SUBJECT_ID" en listes de textes et de temps
grouped_sample_test = filtered_data_test.groupby('SUBJECT_ID').agg({'TEXT': list, 'TIME': list}).reset_index()

## [TEST] Extraire les tokens CLS

Cette partie du code se focalise sur l'extraction des embeddings ClinicalBERT à partir du texte des patients présents dans le jeu de données de test. Les embeddings sont extraits en découpant le texte en parts et en calculant les représentations pour chaque part. Les embeddings de chaque patient sont sauvegardés pour une utilisation ultérieure lors de l'évaluation.

In [ ]:
# Créer un dictionnaire de la forme {n° patient: [liste des textes, valeurs time]} pour faciliter l'itération
grouped_texts_dict = grouped_sample_test.set_index('SUBJECT_ID')[['TEXT', 'TIME']].to_dict(orient='index')

# Créer une liste pour stocker les valeurs de 'TIME' de chaque partie d'un document
time_list_test = []

# Créer un dictionnaire pour stocker les embeddings
embeddings_dict_test = {}

# Parcourir les patients et leurs données textuelles
for subject_id, values in grouped_texts_dict.items():
    texts = values['TEXT']  # Récupérer les documents textuels
    times = values['TIME']  # Récupérer les valeurs de 'TIME' associées

    embeddings_list = []  # Stocker les embeddings de toutes les parties de tous les documents

    # Parcourir les documents et leurs valeurs 'TIME' associées
    for text, time in zip(texts, times):

        # Diviser le texte en parties de longueur appropriée
        encoded_text = tokenizer.encode(text)
        n_tokens = len(encoded_text)
        n_chunks = max(1, n_tokens // 512)
        windows = split_text(text, n_chunks)

        cls_embeddings_list = []  # Stocker les embeddings CLS de chaque partie

        # Parcourir les parties du document
        for window in windows:
            # Convertir la fenêtre en un format compatible avec le modèle
            inputs = tokenizer(window, return_tensors='pt', padding=True, truncation=True, max_length=512).to("cuda")

            with torch.no_grad():
                # Effectuer l'inférence sur le modèle
                outputs = model(**inputs)

            # Récupérer l'embedding du token [CLS] pour chaque fenêtre
            cls_embeddings = outputs.last_hidden_state[:, 0, :]
            cls_embeddings_list.append(cls_embeddings)

            # Ajouter la valeur 'TIME' correspondante à la liste
            time_list_test.append(time)

        # Moyenne des embeddings CLS pour chaque fenêtre
        #avg_cls_embedding = torch.mean(torch.stack(cls_embeddings_list), dim=0)
        #embeddings_list.append(avg_cls_embedding)
        # Ajouter la liste des embeddings CLS à la liste des embeddings pour ce document
        embeddings_list += cls_embeddings_list

    # Stocker les embeddings dans le dictionnaire avec le n° du patient comme clé
    embeddings_dict_test[subject_id] = torch.stack(embeddings_list)


## [TEST] Réduction de dimension

### [TEST] Projection gaussienne aléatoire

Cette partie du code a pour objectif de réduire la dimensionnalité des embeddings extraits lors de l'étape précédente en utilisant une projection gaussienne aléatoire. Le résultat est un ensemble d'embeddings de plus petite dimension qui peut faciliter l'analyse ultérieure.

In [ ]:
# Importer la classe GaussianRandomProjection de la bibliothèque scikit-learn
from sklearn import random_projection

# Créer un nouveau dictionnaire pour stocker les embeddings aplatis
flattened_embeddings_dict_test = {key: tensor.view(tensor.shape[0], -1).cpu().numpy() for key, tensor in embeddings_dict_test.items()}

# Créer une liste pour stocker le nombre d'embeddings pour chaque sujet
lengths_test = [len(tensor) for tensor in flattened_embeddings_dict_test.values()]

# Concaténer tous les embeddings dans une seule matrice
embeddings_np_test = np.concatenate(list(flattened_embeddings_dict_test.values()))

# Initialiser la projection gaussienne aléatoire avec 100 composantes
transformer = random_projection.GaussianRandomProjection(n_components=100)

# Appliquer la projection gaussienne aléatoire sur les embeddings
reduced_embeddings_np_test = transformer.fit_transform(embeddings_np_test)

# Séparer les embeddings transformés pour chaque sujet
reduced_embeddings_list_test = np.split(reduced_embeddings_np_test, np.cumsum(lengths_test)[:-1])

# Recréer le dictionnaire des embeddings transformés avec les clés correspondantes
reduced_embeddings_dict_test = {key: tensor for key, tensor in zip(flattened_embeddings_dict_test.keys(), reduced_embeddings_list_test)}

# Ajouter la valeur 'time' à la fin de chaque embedding réduit
for key, time in zip(reduced_embeddings_dict_test.keys(), time_list_test):
    array_vide = np.empty((reduced_embeddings_dict_test[key].shape[0], 101))
    for i in range(reduced_embeddings_dict_test[key].shape[0]):
        array = np.append(reduced_embeddings_dict_test[key][i], time)
        array_vide[i:] = array
    reduced_embeddings_dict_test[key] = array_vide

# Filtrer les embeddings pour exclure ceux de forme (1, 101)
filtered_embeddings_dict_test = {key: value for key, value in reduced_embeddings_dict_test.items() if value.shape != (1, 101)}
reduced_embeddings_dict_test = filtered_embeddings_dict_test
del filtered_embeddings_dict_test

# Vérifier que le lien entre chaque sujet et ses embeddings est conservé
for key in reduced_embeddings_dict_test.keys():
    print(f"Subject ID: {key}, Embedding shape: {reduced_embeddings_dict_test[key].shape}")
    break

### [TEST] ACP

La technique de l'Analyse en Composantes Principales (ACP) est utilisée pour réduire la dimensionnalité des plongements dans le jeu de données de test, tout en maintenant les informations pertinentes. De plus, cette méthode permet d'afficher la variance expliquée par chaque composante principale.

In [ ]:
from sklearn.decomposition import PCA

# Créer un nouveau dictionnaire pour stocker les embeddings aplatis
flattened_embeddings_dict_test = {key: tensor.view(tensor.shape[0], -1).cpu().numpy() for key, tensor in embeddings_dict_test.items()}

# Créer une liste pour stocker le nombre d'embeddings pour chaque patient
lengths = [len(tensor) for tensor in flattened_embeddings_dict_test.values()]

# Concaténer tous les embeddings dans une seule matrice numpy
embeddings_np_test = np.concatenate(list(flattened_embeddings_dict_test.values()))

# Initialiser l'analyse en composantes principales (ACP) avec 100 composantes
pca = PCA(n_components=100)

# Ajuster le modèle ACP sur les données et les transformer
reduced_embeddings_np_test = pca.fit_transform(embeddings_np_test)

# Séparer les embeddings transformés pour chaque sujet
reduced_embeddings_list_test = np.split(reduced_embeddings_np_test, np.cumsum(lengths)[:-1])

# Recréer le dictionnaire des embeddings transformés avec les clés correspondantes
reduced_embeddings_dict_test = {key: tensor for key, tensor in zip(flattened_embeddings_dict_test.keys(), reduced_embeddings_list_test)}

# Filtrer les embeddings pour exclure ceux de forme (1, 100)
filtered_embeddings_dict_test = {key: value for key, value in reduced_embeddings_dict_test.items() if value.shape != (1, 100)}
reduced_embeddings_dict_test = filtered_embeddings_dict_test
del filtered_embeddings_dict_test

# Ajouter la valeur 'time' à la fin de chaque embedding réduit
for key, time in zip(reduced_embeddings_dict_test.keys(), time_list_test):
    # Créer un tableau vide pour stocker les embeddings et le temps
    array_vide = np.empty((reduced_embeddings_dict_test[key].shape[0], 101))
    for i in range(reduced_embeddings_dict_test[key].shape[0]):
        # Concaténer l'embedding réduit avec la valeur 'time'
        array = np.append(reduced_embeddings_dict_test[key][i], time)
        array_vide[i:] = array
    reduced_embeddings_dict_test[key] = array_vide

# Afficher la variance expliquée par chaque composante principale
print("Variance explained by each component:", pca.explained_variance_ratio_)
print("Total variance explained:", sum(pca.explained_variance_ratio_))

# Vérifier que le lien entre chaque sujet et ses embeddings est bien conservé
for key in reduced_embeddings_dict_test.keys():
    print(f"Subject ID: {key}, Embedding shape: {reduced_embeddings_dict_test[key].shape}")
    break


## [TEST] Calculer les signatures

Ici, le programme calcule les signatures logarithmiques pour les embeddings des données de test. Ces signatures contiennent des informations plus avancées qui peuvent être bénéfiques pour l'entraînement du modèle. Le résultat est un dictionnaire où les embeddings de chaque patient sont représentés sous forme de signatures logarithmiques.

In [ ]:
#import torch
import signatory

# Ordre de la signature tronqué
depth = 2

# Créer un nouveau dictionnaire pour stocker les résultats de la log signature
log_signature_dict_test = {}

# Parcourir le dictionnaire des embeddings réduits pour chaque patient
for key, value in reduced_embeddings_dict_test.items():
    # Convertir le numpy array en un torch tensor et spécifier le type comme float
    tensor = torch.from_numpy(value).float()
    # Alternative : utiliser directement torch.tensor(value, dtype=torch.float)

    # Ajouter une dimension supplémentaire pour correspondre à l'exigence du modèle (batch, stream, channel)
    tensor = tensor.unsqueeze(0)

    # Calculer la log signature en utilisant le modèle signatory
    log_signature = signatory.logsignature(path=tensor, depth=depth)

    # Enlever la dimension batch que nous avons ajoutée précédemment
    log_signature = log_signature.squeeze(0)

    # Ajouter le résultat au dictionnaire avec le numéro du patient comme clé
    log_signature_dict_test[key] = log_signature

# Maintenant, log_signature_dict est un dictionnaire où chaque clé correspond à un patient et chaque valeur est la log signature de ce sujet

## [TEST] Dataframe utilisé pour le test

Enfin, cette partie transforme les résultats de l'analyse en un DataFrame Pandas prêt à être utilisé pour évaluer comment le modèle se comporte sur le jeu de données de test. Ce DataFrame comprend également les étiquettes des patients, ce qui facilite l'évaluation des performances du modèle.

In [ ]:
import pandas as pd

# Convertir le dictionnaire en un DataFrame
df_features_test = pd.DataFrame.from_dict(log_signature_dict_test, orient='index')

# Réinitialiser l'index du DataFrame pour que SUBJECT_ID devienne une colonne
df_features_test.reset_index(inplace=True)

# Renommer la colonne 'index' en 'SUBJECT_ID' pour avoir une colonne de sujet
df_features_test.rename(columns={'index':'SUBJECT_ID'}, inplace=True)

# Parcourir chaque colonne du DataFrame et convertir chaque tenseur en float si nécessaire
for col in df_features_test.columns:
    df_features_test[col] = df_features_test[col].apply(lambda x: x.item() if torch.is_tensor(x) else x)

# Fusionner le DataFrame df_features_test avec le DataFrame new_data_test sur la colonne SUBJECT_ID en utilisant une jointure interne
df_final_test = pd.merge(df_features_test, new_data_test[['SUBJECT_ID', 'HOSPITAL_EXPIRE_FLAG']], on='SUBJECT_ID', how='inner')



In [ ]:
# Afficher le nombre de colonnes du DataFrame final (nombre de caractéristiques + 1 pour la colonne 'HOSPITAL_EXPIRE_FLAG')

df_final_test.shape[1]

# Enregistrement données traités

In [ ]:
import pickle

with open('data/traite/data_train', 'wb') as f1:
    data_train = pickle.dump(df_final, f1)
with open('data/traite/data_test', 'wb') as f1:
    data_test = pickle.dump(df_final_test, f1)

# Importation données pré-traités

In [14]:
import pickle

with open('data/traite/data_train', 'rb') as f1:
    df_final = pickle.load(f1)
with open('data/traite/data_test', 'rb') as f1:
    df_final_test = pickle.load(f1)


In [15]:
import pandas as pd

X_train = df_final.iloc[:, 0:-1]
X_train = pd.merge(X_train, adm, on='SUBJECT_ID', how='left')
X_train['ADMITTIME'] = pd.to_datetime(X_train['ADMITTIME'])
X_train['DISCHTIME'] = pd.to_datetime(X_train['DISCHTIME'])
X_train['DELTATIME'] = (X_train['DISCHTIME'] - X_train['ADMITTIME']).dt.total_seconds() / 3600
X_train_final = X_train.iloc[:, :5152]
X_train_final['DELTATIME'] = X_train['DELTATIME']
print(X_train_final.head(2))

X_test = df_final_test.iloc[:, 0:-1]
X_test = pd.merge(X_test, adm, on='SUBJECT_ID', how='left')
X_test['ADMITTIME'] = pd.to_datetime(X_test['ADMITTIME'])
X_test['DISCHTIME'] = pd.to_datetime(X_test['DISCHTIME'])
X_test['DELTATIME'] = (X_test['DISCHTIME'] - X_test['ADMITTIME']).dt.total_seconds() / 3600
X_test_final = X_test.iloc[:, :5152]
X_test_final['DELTATIME'] = X_test['DELTATIME']

  SUBJECT_ID         0         1         2         3         4         5  \
0      10000  1.458817  4.473032  2.431972  2.288308  0.020622  1.843649   
1       1004  3.737214  7.210907  2.749512  1.521459  1.030078  1.512282   

          6         7         8  ...      5142      5143  5144      5145  \
0 -2.540986 -2.913781  2.027094  ... -0.014916  0.110722   0.0  0.035938   
1 -3.087829  1.846207 -1.026754  ... -0.038609  0.050572   0.0  0.004521   

       5146  5147      5148  5149  5150  DELTATIME  
0 -0.132720   0.0  0.036458   0.0   0.0     499.05  
1 -0.034182   0.0  0.123466   0.0   0.0     368.00  

[2 rows x 5153 columns]


# Reg Logistique

Dans cette partie, nous utilisons la classification par régression logistique pour analyser les données qui ont été préparées à partir des ensembles d'entraînement et de test. Le code commence par configurer un modèle de régression, puis le forme en utilisant les données d'entraînement et effectue des prédictions sur les données de test. Ensuite, il affiche la précision, le rappel et le score F1 du modèle. Cette section nous permet d'évaluer à quel point le modèle de régression logistique prédit avec précision la mortalité à l'hôpital en se basant sur les représentations réduites des données.

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Séparation des données en ensembles d'entraînement et de test

# Sélectionner toutes les colonnes sauf la dernière (la colonne 'HOSPITAL_EXPIRE_FLAG') comme caractéristiques d'entraînement
X_train = df_final.iloc[:, 1:-1]

# Sélectionner la dernière colonne ('HOSPITAL_EXPIRE_FLAG') comme variable cible d'entraînement et la convertir en entiers
y_train = df_final["HOSPITAL_EXPIRE_FLAG"].astype(int)

# Sélectionner les caractéristiques de l'ensemble de test de manière similaire
X_test = df_final_test.iloc[:, 1:-1]
y_test = df_final_test["HOSPITAL_EXPIRE_FLAG"].astype(int)

# Initialisation du modèle de régression logistique

# Créer une instance du modèle de régression logistique avec les hyperparamètres spécifiés
model = LogisticRegression(penalty='l1', solver='saga', C=0.1, max_iter=1000)

# Entraînement du modèle

# Entraîner le modèle sur les données d'entraînement
model.fit(X_train, y_train)

# Prédire les étiquettes sur l'ensemble de test en utilisant le modèle entraîné
y_pred = model.predict(X_test)

# Évaluation des performances du modèle

# Calculer et afficher l'accuracy du modèle
accuracy = accuracy_score(y_test, y_pred)
print("Précision: %.2f%%" % (accuracy * 100.0))

# Calculer et afficher le rappel du modèle
recall = recall_score(y_test, y_pred)
print("Rappel: %.2f%%" % (recall * 100.0))

# Calculer et afficher le F1-score du modèle
f1 = f1_score(y_test, y_pred)
print("F1-score: %.2f%%" % (f1 * 100.0))

KeyboardInterrupt: 

## Prédiction conforme méthode split conformal + small sets pour régression logistique

In [11]:
import numpy as np

# Problem setup
n = 500 # number of calibration points
alpha = 0.1 # 1-alpha is the desired coverage
# Split the softmax scores into calibration and validation sets (save the shuffling)
smx = model.predict_proba(X_test)
idx = np.array([1] * n + [0] * (smx.shape[0]-n)) > 0
np.random.shuffle(idx)
# ix est un vecteur de taille 50 000 (nb d'images) qui contient n True et 50 000 - n False disposé aléatoirement
cal_smx, val_smx = smx[idx,:], smx[~idx,:]
cal_labels, val_labels = y_test[idx], y_test[~idx]

# 1: get conformal scores. n = calib_Y.shape[0]
cal_scores = 1-cal_smx[np.arange(n),cal_labels]
# Pour chacunes des images du set de calibration, score de conformité = 1-softmax associé au vrai label (liste de n éléments)
# score de conformité élevé quand softmax faible = prédiction du model mauvaise

# 2: get adjusted quantile
q_level = np.ceil((n+1)*(1-alpha))/n
qhat = np.quantile(cal_scores, q_level, method='higher') # valeur du 9 ème décile environ

# 3: form prediction sets
prediction_sets = val_smx >= (1-qhat)
# Pour chaque image du set de validation, on garde les labels dont le softmax dépasse le seuil (on remplace les valeurs par True/False) 

# Calculate empirical coverage
empirical_coverage = prediction_sets[np.arange(prediction_sets.shape[0]),val_labels].mean()
# On regarde en moyenne si le set de prédiction contient bien le vrai label
print(f"The empirical coverage is: {empirical_coverage}")

for i in range(10):
    prediction_set = smx[i] > 1-qhat
    label_strings = ["0","1"]
    filtered_labels = [label for keep, label in zip(list(prediction_set), label_strings) if keep]
    print(f"The prediction set is: {filtered_labels}")
    print(f"The real value is: {y_test[i]}")

The empirical coverage is: 0.9160401002506265
The prediction set is: ['0', '1']
The real value is: 0
The prediction set is: ['0', '1']
The real value is: 0
The prediction set is: ['0', '1']
The real value is: 0
The prediction set is: ['0', '1']
The real value is: 0
The prediction set is: ['0', '1']
The real value is: 0
The prediction set is: ['0', '1']
The real value is: 0
The prediction set is: ['1']
The real value is: 1
The prediction set is: ['0', '1']
The real value is: 0
The prediction set is: ['1']
The real value is: 1
The prediction set is: ['1']
The real value is: 0


## Prédiction conforme méthode split conformal + meilleur méthode ? pour régression logistique

In [12]:
import numpy as np

# Problem setup
n = 500 # number of calibration points
alpha = 0.1 # 1-alpha is the desired coverage
# Split the softmax scores into calibration and validation sets (save the shuffling)
smx = model.predict_proba(X_test)

# Set RAPS regularization parameters (larger lam_reg and smaller k_reg leads to smaller sets)
lam_reg = 0.01
k_reg = 1
disallow_zero_sets = False # Set this to False in order to see the coverage upper bound hold
rand = True # Set this to True in order to see the coverage upper bound hold
reg_vec = np.array(k_reg*[0,] + (smx.shape[1]-k_reg)*[lam_reg,])[None,:]


idx = np.array([1] * n + [0] * (smx.shape[0]-n)) > 0
np.random.shuffle(idx)
# ix est un vecteur de taille 50 000 (nb d'images) qui contient n True et 50 000 - n False disposé aléatoirement
cal_smx, val_smx = smx[idx,:], smx[~idx,:]
cal_labels, val_labels = y_test[idx], y_test[~idx]

# Get scores. calib_X.shape[0] == calib_Y.shape[0] == n
cal_pi = cal_smx.argsort(1)[:,::-1]; 
print(cal_pi.shape)
cal_srt = np.take_along_axis(cal_smx,cal_pi,axis=1)
print(cal_srt.shape)
cal_srt_reg = cal_srt + reg_vec
cal_L = np.where(cal_pi == cal_labels.values[:, None])[1]
cal_scores = cal_srt_reg.cumsum(axis=1)[np.arange(n),cal_L] - np.random.rand(n)*cal_srt_reg[np.arange(n),cal_L]
# Get the score quantile
qhat = np.quantile(cal_scores, np.ceil((n+1)*(1-alpha))/n, interpolation='higher')
# Deploy
n_val = val_smx.shape[0]
val_pi = val_smx.argsort(1)[:,::-1]
val_srt = np.take_along_axis(val_smx,val_pi,axis=1)
val_srt_reg = val_srt + reg_vec
val_srt_reg_cumsum = val_srt_reg.cumsum(axis=1)
indicators = (val_srt_reg.cumsum(axis=1) - np.random.rand(n_val,1)*val_srt_reg) <= qhat if rand else val_srt_reg.cumsum(axis=1) - val_srt_reg <= qhat
if disallow_zero_sets: indicators[:,0] = True
prediction_sets = np.take_along_axis(indicators,val_pi.argsort(axis=1),axis=1)


# Calculate empirical coverage
empirical_coverage = prediction_sets[np.arange(n_val),val_labels].mean()
# On regarde en moyenne si le set de prédiction contient bien le vrai label
print(f"The empirical coverage is: {empirical_coverage}")

for i in range(10):
    prediction_set = smx[i] > 1-qhat
    label_strings = ["0","1"]
    filtered_labels = [label for keep, label in zip(list(prediction_set), label_strings) if keep]
    print(f"The prediction set is: {filtered_labels}")
    print(f"The real value is: {y_test[i]}")

(200, 2)
(200, 2)
The empirical coverage is: 0.8822055137844611
The prediction set is: ['0', '1']
The real value is: 0
The prediction set is: ['0', '1']
The real value is: 0
The prediction set is: ['0', '1']
The real value is: 0
The prediction set is: ['0', '1']
The real value is: 0
The prediction set is: ['0', '1']
The real value is: 0
The prediction set is: ['0', '1']
The real value is: 0
The prediction set is: ['0', '1']
The real value is: 1
The prediction set is: ['0', '1']
The real value is: 0
The prediction set is: ['0', '1']
The real value is: 1
The prediction set is: ['0', '1']
The real value is: 0


## Prédiction conforme méthode full conformal + small sets pour régression logistique

In [13]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.exceptions import ConvergenceWarning
import warnings

# Ignorer les avertissements de convergence
warnings.filterwarnings("ignore", category=ConvergenceWarning)

model = LogisticRegression(penalty='l1', solver='saga', C=0.1, max_iter=1000)

def full_conformal(k):
    alpha = 0.1 # 1-alpha is the desired coverage
    X = pd.concat([X_train, X_test.iloc[k:k+1]])
    y = y_train.copy() 
    n = len(y_train)
    prediction = []
    for i in range(2):
        y.loc[n] = i
        model.fit(X, y)
        smx = model.predict_proba(X)
        scores = 1-smx[np.arange(n+1), y]
        q_level = np.ceil((n+1)*(1-alpha))/n
        qhat = np.quantile(scores[:-1], q_level, method='higher') # valeur du 9 ème décile environ
        if scores[-1] <= qhat:
            prediction.append(str(i))
    return prediction

for i in range(5):
    prediction_set = full_conformal(i)
    print(f"The prediction set is: {prediction_set}")
    print(f"The real value is: {y_test[i]}")

The prediction set is: ['0', '1']
The real value is: 0
The prediction set is: ['0']
The real value is: 0
The prediction set is: ['1']
The real value is: 0
The prediction set is: ['0']
The real value is: 0
The prediction set is: ['1']
The real value is: 0


# Random Forest

Dans cette partie, nous utilisons un modèle de classification de forêt aléatoire pour estimer la probabilité de décès à l'hôpital. Nous optimisons les hyperparamètres en utilisant GridSearchCV afin de trouver la meilleure configuration pour le nombre maximal de caractéristiques (max_features). Les performances du modèle sont évaluées selon la précision, le rappel et le score F1.

In [17]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, f1_score, mean_squared_error, make_scorer
import math

Pb_classifcation = False

# Séparation des données en ensembles d'entraînement et de test

if Pb_classifcation:
    # Sélectionner toutes les colonnes sauf la dernière (la colonne 'HOSPITAL_EXPIRE_FLAG') comme caractéristiques d'entraînement
    X_train = df_final.iloc[:, 1:-1]

    # Sélectionner la dernière colonne ('HOSPITAL_EXPIRE_FLAG') comme variable cible d'entraînement et la convertir en entiers
    y_train = df_final["HOSPITAL_EXPIRE_FLAG"].astype(int)

    # Sélectionner les caractéristiques de l'ensemble de test de manière similaire
    X_test = df_final_test.iloc[:, 1:-1]
    y_test = df_final_test["HOSPITAL_EXPIRE_FLAG"].astype(int)

else:
    X_train = X_train_final.iloc[:, 1:-1]
    y_train = X_train_final['DELTATIME']

    X_test = X_test_final.iloc[:, 1:-1]
    y_test = X_test_final['DELTATIME']


# Définir les valeurs des hyperparamètres à tester

# Créer un dictionnaire de valeurs à tester pour le nombre maximal de caractéristiques utilisées à chaque division de l'arbre
param_grid = {
    'max_features': list(range(30, 121, 10))
}

# Initialisation du modèle Random Forest

# Créer une instance du modèle Random Forest avec des hyperparamètres spécifiés
if Pb_classifcation:
    model = RandomForestClassifier(criterion='entropy', n_estimators=150)
    # Recherche des meilleurs hyperparamètres

    # Créer un objet GridSearchCV pour effectuer une recherche des meilleurs hyperparamètres
    # cv=5 indique une validation croisée en 5 plis et scoring='f1_micro' utilise le F1-score pour l'évaluation
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1_micro')
else:
    model = RandomForestRegressor(criterion='squared_error', n_estimators=150)
    # Définir la métrique MSE comme score
    mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring=mse_scorer)


# Effectuer la recherche des meilleurs hyperparamètres en utilisant les données d'entraînement
grid_search.fit(X_train, y_train)


# Obtenir les meilleures valeurs des hyperparamètres
best_max_features = grid_search.best_params_['max_features']

# Utilisation du modèle avec les meilleurs hyperparamètres pour la prédiction sur l'ensemble de test

# Créer une nouvelle instance du modèle Random Forest avec les meilleurs hyperparamètres trouvés
if Pb_classifcation:
    best_model = RandomForestClassifier(criterion='entropy', n_estimators=150, max_features=best_max_features)
else:
    best_model = RandomForestRegressor(criterion='squared_error', n_estimators=150, max_features=best_max_features)

# Entraîner le modèle avec les meilleurs hyperparamètres sur les données d'entraînement
best_model.fit(X_train, y_train)

# Prédire les étiquettes sur l'ensemble de test en utilisant le modèle entraîné
y_pred = best_model.predict(X_test)

# Évaluation des performances du modèle
if Pb_classifcation:

    # Calculer et afficher l'accuracy du modèle
    accuracy = accuracy_score(y_test, y_pred)
    print("Précision: %.2f%%" % (accuracy * 100.0))

    # Calculer et afficher le rappel du modèle
    recall = recall_score(y_test, y_pred)
    print("Rappel: %.2f%%" % (recall * 100.0))

    # Calculer et afficher le F1-score du modèle
    f1 = f1_score(y_test, y_pred)
    print("F1-score: %.2f%%" % (f1 * 100.0))
else:
    mse = mean_squared_error(y_test, y_pred)
    print("MSE sur les données d'entraînement:", mse)

MSE sur les données d'entraînement: 89726.01388893234


# Random Forest sur la valeur absolue des résidus

In [21]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
import math

# Séparation des données en ensembles d'entraînement et de test

# Sélectionner la dernière colonne ('HOSPITAL_EXPIRE_FLAG') comme variable cible d'entraînement
r_train = abs(X_test_final['DELTATIME'] - y_pred)

# Définir les valeurs des hyperparamètres à tester

# Créer un dictionnaire de valeurs à tester pour le nombre maximal de caractéristiques utilisées à chaque division de l'arbre
param_grid = {
    'max_features': list(range(30, 121, 10))
}

# Initialisation du modèle Random Forest

# Créer une nouvelle instance du modèle Random Forest avec les meilleurs hyperparamètres trouvés
best_model = RandomForestRegressor(criterion='squared_error', n_estimators=150, max_features=best_max_features)

# Entraîner le modèle avec les meilleurs hyperparamètres sur les données d'entraînement
best_model.fit(X_test, r_train)

# Prédire les étiquettes sur l'ensemble de test en utilisant le modèle entraîné
r_pred = best_model.predict(X_test)

## Prédiction conforme méthode split conformal + small sets pour random  forest

In [ ]:
import numpy as np

# Problem setup
n = 500 # number of calibration points
alpha = 0.1 # 1-alpha is the desired coverage
# Split the softmax scores into calibration and validation sets (save the shuffling)
smx = best_model.predict_proba(X_test)
idx = np.array([1] * n + [0] * (smx.shape[0]-n)) > 0
np.random.shuffle(idx)
# ix est un vecteur de taille 50 000 (nb d'images) qui contient n True et 50 000 - n False disposé aléatoirement
cal_smx, val_smx = smx[idx,:], smx[~idx,:]
cal_labels, val_labels = y_test[idx], y_test[~idx]

# 1: get conformal scores. n = calib_Y.shape[0]
cal_scores = 1-cal_smx[np.arange(n),cal_labels]
# Pour chacunes des images du set de calibration, score de conformité = 1-softmax associé au vrai label (liste de n éléments)
# score de conformité élevé quand softmax faible = prédiction du model mauvaise

# 2: get adjusted quantile
q_level = np.ceil((n+1)*(1-alpha))/n
qhat = np.quantile(cal_scores, q_level, method='higher') # valeur du 9 ème décile environ

# 3: form prediction sets
prediction_sets = val_smx >= (1-qhat)
# Pour chaque image du set de validation, on garde les labels dont le softmax dépasse le seuil (on remplace les valeurs par True/False) 

# Calculate empirical coverage
empirical_coverage = prediction_sets[np.arange(prediction_sets.shape[0]),val_labels].mean()
# On regarde en moyenne si le set de prédiction contient bien le vrai label
print(f"The empirical coverage is: {empirical_coverage}")

for i in range(10):
    prediction_set = smx[i] > 1-qhat
    label_strings = ["0","1"]
    filtered_labels = [label for keep, label in zip(list(prediction_set), label_strings) if keep]
    print(f"The prediction set is: {filtered_labels}")
    print(f"The real value is: {y_test[i]}")

## Prédiction conforme méthode split conformal + meilleur méthode ? pour random  forest

In [ ]:
import numpy as np

# Problem setup
n = 500 # number of calibration points
alpha = 0.1 # 1-alpha is the desired coverage
# Split the softmax scores into calibration and validation sets (save the shuffling)
smx = best_model.predict_proba(X_test)

# Set RAPS regularization parameters (larger lam_reg and smaller k_reg leads to smaller sets)
lam_reg = 0.01
k_reg = 1
disallow_zero_sets = False # Set this to False in order to see the coverage upper bound hold
rand = True # Set this to True in order to see the coverage upper bound hold
reg_vec = np.array(k_reg*[0,] + (smx.shape[1]-k_reg)*[lam_reg,])[None,:]


idx = np.array([1] * n + [0] * (smx.shape[0]-n)) > 0
np.random.shuffle(idx)
# ix est un vecteur de taille 50 000 (nb d'images) qui contient n True et 50 000 - n False disposé aléatoirement
cal_smx, val_smx = smx[idx,:], smx[~idx,:]
cal_labels, val_labels = y_test[idx], y_test[~idx]

# Get scores. calib_X.shape[0] == calib_Y.shape[0] == n
cal_pi = cal_smx.argsort(1)[:,::-1]; 
print(cal_pi.shape)
cal_srt = np.take_along_axis(cal_smx,cal_pi,axis=1)
print(cal_srt.shape)
cal_srt_reg = cal_srt + reg_vec
cal_L = np.where(cal_pi == cal_labels.values[:, None])[1]
cal_scores = cal_srt_reg.cumsum(axis=1)[np.arange(n),cal_L] - np.random.rand(n)*cal_srt_reg[np.arange(n),cal_L]
# Get the score quantile
qhat = np.quantile(cal_scores, np.ceil((n+1)*(1-alpha))/n, interpolation='higher')
# Deploy
n_val = val_smx.shape[0]
val_pi = val_smx.argsort(1)[:,::-1]
val_srt = np.take_along_axis(val_smx,val_pi,axis=1)
val_srt_reg = val_srt + reg_vec
val_srt_reg_cumsum = val_srt_reg.cumsum(axis=1)
indicators = (val_srt_reg.cumsum(axis=1) - np.random.rand(n_val,1)*val_srt_reg) <= qhat if rand else val_srt_reg.cumsum(axis=1) - val_srt_reg <= qhat
if disallow_zero_sets: indicators[:,0] = True
prediction_sets = np.take_along_axis(indicators,val_pi.argsort(axis=1),axis=1)


# Calculate empirical coverage
empirical_coverage = prediction_sets[np.arange(n_val),val_labels].mean()
# On regarde en moyenne si le set de prédiction contient bien le vrai label
print(f"The empirical coverage is: {empirical_coverage}")

for i in range(10):
    prediction_set = smx[i] > 1-qhat
    label_strings = ["0","1"]
    filtered_labels = [label for keep, label in zip(list(prediction_set), label_strings) if keep]
    print(f"The prediction set is: {filtered_labels}")
    print(f"The real value is: {y_test[i]}")

## Prédiction conforme pb regression méthode split conformal pour random  forest

In [30]:
import numpy as np

# Problem setup
n = 500 # number of calibration points
alpha = 0.1 # 1-alpha is the desired coverage
# Split the softmax scores into calibration and validation sets (save the shuffling)
smx = y_pred
idx = np.array([1] * n + [0] * (smx.shape[0]-n)) > 0
np.random.shuffle(idx)
# ix est un vecteur de taille 50 000 (nb d'images) qui contient n True et 50 000 - n False disposé aléatoirement
cal_smx, val_smx = smx[idx], smx[~idx]
cal_value, val_value = y_test[idx], y_test[~idx]
cal_residual, val_residual = r_pred[idx], r_pred[~idx]

# cal_smx=E(Y|X), and cal_residual dans le même principe que stddev(Y|X)

# 1: get conformal scores. 
scores = abs(cal_value-cal_smx)/cal_residual
# score de conformité élevé quand softmax faible = prédiction du model mauvaise

# 2: get adjusted quantile
q_level = np.ceil((n+1)*(1-alpha))/n
qhat = np.quantile(scores, q_level, method='higher')

prediction_sets = (val_smx-val_residual*qhat, val_smx+val_residual*qhat)
# Pour chaque image du set de validation, on garde les labels dont le softmax dépasse le seuil (on remplace les valeurs par True/False) 
occurrences_correctes = 0

# Parcourir chaque intervalle dans prediction_sets
for i in range(len(prediction_sets[0])):
    interval_min = prediction_sets[0][i]
    interval_max = prediction_sets[1][i]
    print(f"The prediction set is: [{interval_min}, {interval_max}]")
    print(f"The real value is: {y_test[i]}")
    
    # Vérifier si la valeur réelle est contenue dans l'intervalle
    if interval_min <= y_test[i] <= interval_max:
        occurrences_correctes += 1

# Calculer le pourcentage d'occurrences correctes
pourcentage_correct = (occurrences_correctes / len(prediction_sets[0])) * 100
print(pourcentage_correct)

The prediction set is: [66.70690415908646, 263.6053217668391]
The real value is: 44.18333333333333
The prediction set is: [49.706986681346905, 184.74454665198635]
The real value is: 100.6
The prediction set is: [138.3640801610291, 288.4833153416164]
The real value is: 229.55
The prediction set is: [103.3240094794337, 416.92687940945507]
The real value is: 138.16666666666666
The prediction set is: [126.95601395322757, 506.46542784571403]
The real value is: 65.68333333333334
The prediction set is: [314.2439863423969, 514.0586739750634]
The real value is: 248.68333333333334
The prediction set is: [95.56332050331224, 534.9509017189098]
The real value is: 136.51666666666668
The prediction set is: [185.73237942014637, 467.0364909502239]
The real value is: 170.35
The prediction set is: [58.59459507549274, 128.7695382578404]
The real value is: 530.6666666666666
The prediction set is: [227.4480492322348, 570.6667791628272]
The real value is: 400.93333333333334
The prediction set is: [227.448049

## Prédiction conforme méthode full conformal + small sets pour random forest

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

model = RandomForestClassifier(criterion='entropy', n_estimators=150)

param_grid = {
    'max_features': list(range(30, 121, 10))
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1_micro')

# Effectuer la recherche des meilleurs hyperparamètres en utilisant les données d'entraînement
grid_search.fit(X_train, y_train)

# Obtenir les meilleures valeurs des hyperparamètres
best_max_features = grid_search.best_params_['max_features']

# Utilisation du modèle avec les meilleurs hyperparamètres pour la prédiction sur l'ensemble de test

# Créer une nouvelle instance du modèle Random Forest avec les meilleurs hyperparamètres trouvés
model = RandomForestClassifier(criterion='entropy', n_estimators=150, max_features=best_max_features)


def full_conformal(k):
    alpha = 0.1 # 1-alpha is the desired coverage
    X = pd.concat([X_train, X_test.iloc[k:k+1]])
    y = y_train.copy() 
    n = len(y_train)
    prediction = []
    for i in range(2):
        y.loc[n] = i
        model.fit(X, y)
        smx = model.predict_proba(X)
        scores = 1-smx[np.arange(n+1), y]
        q_level = np.ceil((n+1)*(1-alpha))/n
        qhat = np.quantile(scores[:-1], q_level, method='higher') # valeur du 9 ème décile environ
        if scores[-1] <= qhat:
            prediction.append(str(i))
    return prediction

for i in range(10):
    prediction_set = full_conformal(i)
    print(f"The prediction set is: {prediction_set}")
    print(f"The real value is: {y_test[i]}")

# Sauvegarder les données

Nous procédons à une sauvegarde des données en enregistrant les plongements traités ainsi que les valeurs temporelles dans des fichiers JSON et Pickle, afin de pouvoir les utiliser ou les analyser ultérieurement.

In [ ]:
import json

# Créer un nouveau dictionnaire pour stocker les embeddings aplatis convertis en listes
flattened_embeddings_dict_converted = {}

# Parcourir le dictionnaire flattened_embeddings_dict_test
for clé, valeur in flattened_embeddings_dict_test.items():
    # Convertir chaque valeur (numpy array) en liste et stocker dans le nouveau dictionnaire
    flattened_embeddings_dict_converted[clé] = valeur.tolist()

# Sauvegarder le dictionnaire converti en tant que fichier JSON

# Spécifier le chemin du fichier JSON où vous souhaitez enregistrer les données
with open('embeddings_dict_test.json', 'w') as fichier:
    # Utiliser la bibliothèque JSON pour écrire le dictionnaire converti dans le fichier JSON
    json.dump(flattened_embeddings_dict_converted, fichier)

In [ ]:
# Importer le module pour la serialization des objets
import pickle

# Sauvegarder la liste dans un fichier binaire (pickle)
with open('time_list_test.pkl', 'wb') as f:
    # Utiliser la bibliothèque pickle pour écrire la liste dans le fichier binaire
    pickle.dump(time_list_test, f)


In [ ]:
!pip install nbconvert 